In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import tree
from sklearn.tree import _tree
import pydotplus
from IPython.display import Image, display, clear_output
from ipywidgets import widgets, Layout
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from bokeh.plotting import *
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC, LinearSVC
from tabulate import tabulate
import squarify
import spacy
import string
from nltk.classify import SklearnClassifier
from subprocess import check_output
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import string
import base64
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import holoviews as hv
import copy
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from time import time
stopwords = stopwords.words('english')
sns.set_context('talk')

import re
#spacy.load('en')
from spacy.lang.en import English
parser = English()
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfTransformer
import tabulate
from sklearn.metrics import confusion_matrix
from spacy import displacy
sns.set_context("talk")

In [2]:
episodes= ["SW_EpisodeIV.txt","SW_EpisodeV.txt", "SW_EpisodeVI.txt"]
SW= []
star_all = pd.DataFrame()
for episode in episodes:
    star_1 = pd.read_csv(episode, sep='" "', engine='python')
    star_1 = star_1.loc[:,star_1.dtypes==object].apply(lambda star_1:star_1.str.replace('"', ""))
    star_1 = star_1.reset_index(drop=True)
    star_1.columns = ['character', 'dialogue']
    SW.append(star_1)
star_all = pd.concat(SW)
star_all = star_all.reset_index(drop=True)
star_all.shape

(2523, 2)

In [3]:
classes=star_all.character.value_counts()
classes_big=classes[classes>=40]
sufficient=star_all[star_all.character.isin(classes_big.index)][["character",'dialogue']]

In [4]:
sufficient=sufficient.reset_index(drop=True)
sufficient.shape

(1930, 2)

In [5]:
sufficient["tokens"]=sufficient['dialogue'].apply(word_tokenize) 

In [6]:
def no_punct(tokens):
    words = [word for word in tokens if word.isalpha()]
    return words
def to_lower(tokens):
    words = [w.lower() for w in tokens]
    return words

In [7]:
sufficient["tokens"]=sufficient['tokens'].apply(to_lower)
sufficient["tokens"]=sufficient['tokens'].apply(no_punct)

In [8]:
nlp = spacy.load('en_core_web_sm')
punctuations = string.punctuation

def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [9]:
#the order of words is been taken into account
import spacy
nlp = spacy.load('en_core_web_sm')
a = nlp('the test sentence')
b = nlp('the sentence test')
print(a.vector[:4])
print(b.vector[:4])
print(a.similarity(b))

[0.66365343 1.5335983  0.8064194  2.6753042 ]
[0.745111  1.931929  0.7636134 2.2745903]
0.9531291379577567


In [10]:
sufficient["words"]=sufficient.tokens.apply(lambda x: len(x))

In [11]:
nlp = spacy.load('en_core_web_sm')
punctuations = string.punctuation

def cleanup_text(docs, logging=False):
    texts = []
    counter = 1
    for doc in docs:
        if counter % 1000 == 0 and logging:
            print("Processed %d out of %d documents." % (counter, len(docs)))
        counter += 1
        doc = nlp(doc, disable=['parser', 'ner'])
        tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
        tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
        tokens = ' '.join(tokens)
        texts.append(tokens)
    return pd.Series(texts)

In [12]:
def evaluate(tokenizer, textcat, texts, cats):
    #changed to "argmax" of the scores
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]["cats"]
        
        #here: set label and score to the highest of the predictions. Compare to truth
        #Recall basically deactivated
        label, score = sorted(doc.cats.items(), key=lambda val: val[1], reverse=True)[0]
        #for label, score in doc.cats.items():
        if gold[label] >= 0.5:
            tp += 1.
        elif gold[label] < 0.5:
            fp += 1.
        #elif score < 0.5 and gold[label] < 0.5:
        #    tn += 1
        #elif score < 0.5 and gold[label] >= 0.5:
        #    fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}


# spaCy with blank model

In [17]:
def clean_string(mystring):
    return re.sub('[^A-Za-z\ 0-9 ]+', '', mystring)




model=None
output_dir=""
n_iter=10
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

# add the text classifier to the pipeline if it doesn't exist
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'textcat' not in nlp.pipe_names:
    textcat = nlp.create_pipe('textcat')
    nlp.add_pipe(textcat, last=True)
# otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe('textcat')

# add label to text classifier
for i in ['THREEPIO', 'VADER', 'LUKE', 'BEN', 'LEIA', 'HAN', 'EMPEROR','YODA', 'LANDO']: #hier standen emotionen
    textcat.add_label(i)


df = sufficient

character_values = df['character'].unique()
labels_default = dict((v, 0) for v in character_values)

train_data = []
dev_data = []
dev_texts = []
dev_cats = []
data_all=[]
for i, row in df.iterrows():

    label_values = copy.deepcopy(labels_default)
    label_values[row['character']] = 1

    data_all.append(((clean_string(row['dialogue'])), {"cats": label_values}))


train_data ,dev_data = train_test_split(data_all,test_size=0.2)     


for i in range(len(dev_data)):
    dev_texts.append(dev_data[i][0])#dialogue
    dev_cats.append(dev_data[i][1])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
nlp.vocab.vectors.name = 'spacy_pretrained_vectors'
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print('{:^5}\t{:^5}'.format('LOSS', 'P'))
    for i in range(n_iter):
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}'  # print a simple table
              .format(losses['textcat'], scores['textcat_p']))


# test the trained model
test_text = "I am your father"
doc = nlp(test_text)
print(test_text, sorted(doc.cats.items(), key=lambda val: val[1], reverse=True))

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)


Created blank 'en' model
Training the model...
LOSS 	  P  
426.509	0.251
319.878	0.316
269.537	0.352
240.000	0.404
210.097	0.433
185.652	0.430
170.108	0.427
151.407	0.435
137.181	0.438
127.999	0.409
I am your father [('LUKE', 0.7716049551963806), ('VADER', 0.05292923375964165), ('LEIA', 0.05280479043722153), ('BEN', 0.03298528492450714), ('LANDO', 0.02779574505984783), ('THREEPIO', 0.02031521685421467), ('HAN', 0.015233706682920456), ('EMPEROR', 0.011022736318409443), ('YODA', 0.0011719156755134463)]
Saved model to .


# spaCy with pretrained model

In [18]:
def clean_string(mystring):
    return re.sub('[^A-Za-z\ 0-9 ]+', '', mystring)




model="en_core_web_sm" #included word-vectors
output_dir=""
n_iter=10
if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

# add the text classifier to the pipeline if it doesn't exist
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'textcat' not in nlp.pipe_names:
    textcat = nlp.create_pipe('textcat')
    nlp.add_pipe(textcat, last=True)
# otherwise, get it, so we can add labels to it
else:
    textcat = nlp.get_pipe('textcat')

# add label to text classifier
for i in ['THREEPIO', 'VADER', 'LUKE', 'BEN', 'LEIA', 'HAN', 'EMPEROR','YODA', 'LANDO']: #hier standen emotionen
    textcat.add_label(i)


df = sufficient


character_values = df['character'].unique()
labels_default = dict((v, 0) for v in character_values)

train_data = []
dev_data = []
dev_texts = []
dev_cats = []
data_all=[]
for i, row in df.iterrows():

    label_values = copy.deepcopy(labels_default)
    label_values[row['character']] = 1

    data_all.append(((clean_string(row['dialogue'])), {"cats": label_values}))
    
train_data ,dev_data = train_test_split(data_all,test_size=0.2)     

for i in range(len(dev_data)):
    dev_texts.append(dev_data[i][0])#dialogue
    dev_cats.append(dev_data[i][1])

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
nlp.vocab.vectors.name = 'spacy_pretrained_vectors'
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()
    print("Training the model...")
    print('{:^5}\t{:^5}'.format('LOSS', 'P'))
    for i in range(n_iter):
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}'  # print a simple table
              .format(losses['textcat'], scores['textcat_p']))


# test the trained model
test_text = "I am your father"
doc = nlp(test_text)
print(test_text, sorted(doc.cats.items(), key=lambda val: val[1], reverse=True))

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)


Loaded model 'en_core_web_sm'
Training the model...
LOSS 	  P  
421.440	0.303
304.903	0.326
262.564	0.355
225.038	0.391
197.946	0.381
174.963	0.394
159.480	0.407
144.166	0.399
131.665	0.407
115.822	0.407
I am your father [('LUKE', 0.9127466082572937), ('LEIA', 0.028731418773531914), ('VADER', 0.023627549409866333), ('BEN', 0.013822007924318314), ('EMPEROR', 0.008743206039071083), ('LANDO', 0.005537025630474091), ('THREEPIO', 0.0024341237731277943), ('HAN', 0.0021043228916823864), ('YODA', 0.0008121429709717631)]
Saved model to .
